In [ ]:
#IMPORT THE FOLLOWING

import cv2
import numpy as np
import os
from random import shuffle
import matplotlib.pyplot as plt

DATADIR = "E:/DBR project2" #GIVE YOUR OWN PATH
CATEGORIES = ["running" , "sitting"] #CHANGE CATEGORIES


for category in CATEGORIES:
    path = os.path.join(DATADIR,category) #joins our path to cat and dog directory
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
        plt.imshow(img_array , cmap="gray")
        plt.show()
        break
    break

print(img_array.shape) #OPTIONAL

IMG_SIZE = 100
new_array= cv2.resize(img_array , (IMG_SIZE , IMG_SIZE))
plt.imshow(new_array , cmap='gray')
plt.show()

training_data=[]
def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR,category) #joins our path to cat and dog directory
        class_num= CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                new_array= cv2.resize(img_array , (IMG_SIZE , IMG_SIZE))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass
create_training_data()

#TO SEE THE LENGHTH OF TRAINING DATA
print(len(training_data))

#SHUFFLE THE TRAINING DATA
import random

random.shuffle(training_data)

#FOR CHECKING THE TWO LABELS 0 FOR TONGUE AND 1 FOR CLOSED
for sample in training_data[:20]:
    print(sample[1])

#CREATE TWO PICKLE FILES
X = []
y = []
for features , label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
import pickle

pickle_out=open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out=open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

y[5]

In [1]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time


X= pickle.load(open("X.pickle","rb"))
y= pickle.load(open("y.pickle","rb"))

X=X/255.0

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)
            
            model = Sequential()
            
            model.add(Conv2D(layer_size ,(3,3) , input_shape=X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))

            model.add(Conv2D(conv_layer ,(3,3)))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))

            model.add(Flatten())
            
            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))
            
            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss="binary_crossentropy",
                         optimizer="adam",
                         metrics=['accuracy'])

            model.fit(X,y,batch_size=10 ,epochs=32, validation_split=0.1)
    
model.save('64x3-CNN.model')

3-conv-64-nodes-0-dense-1582527699
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2997 samples, validate on 333 samples
Epoch 1/32
2997/2997 [==============================] - 33s 11ms/sample - loss: 0.4878 - acc: 0.7644 - val_loss: 0.3453 - val_acc: 0.8919
Epoch 2/32
2997/2997 [==============================] - 30s 10ms/sample - loss: 0.3264 - acc: 0.8795 - val_loss: 0.3601 - val_acc: 0.8288
Epoch 3/32
2997/2997 [==============================] - 30s 10ms/sample - loss: 0.2530 - acc: 0.9022 - val_loss: 0.2557 - val_acc: 0.9189
Epoch 4/32
2997/2997 [==============================] - 29s 10ms/sample - loss: 0.1896 - acc: 0.9326 - val_loss: 0.1991 - val_acc: 0.9279
Epoch 5/32
2997/2997 [==============================] - 29s 10ms/sample - loss: 0.1294 - acc: 0.9570 - val_loss: 0.1913 - val_acc: 0.9369
Epoch 6/32
2997/2997 [==============================]

In [17]:
#TESTING WITH AN IMAGE THAT IS NOT IN THE DATASET

import cv2
import tensorflow as tf

CATEGORIES = ["running" , "sitting" ] # will use this to convert prediction num to string value
filepath = "E:/d.jpg" #GIVE PATH OF YOUR IMAGE
def prepare(filepath):
    IMG_SIZE = 100  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # return the image with shaping that TF wants.

#SAVE THE MODEL
model = tf.keras.models.load_model("64x3-CNN.model")

#FOLLOWING WILL GIVE YOU PREDICTION
prediction = model.predict([prepare(filepath)])

print(CATEGORIES[int(prediction[0][0])])

running
